In [17]:

# preprocess chatbot survey -> prompt x condition x scores
import io
import json
import re
from pathlib import Path
import numpy as np
import pandas as pd

SURVEY_FILE = Path("Answers_masked.csv")
BLOCKMAP_FILE = Path("surveyblock_map.csv")
TASKS_FILE = Path("tasks_data.json")
OUT_WIDE = Path("processed_scores.csv")
OUT_LONG = Path("answers_long.csv")
OUT_XLSX = Path("processed_scores.xlsx")

def detect_sep(first_line: str) -> str:
    # pick ; if there are more semicolons than commas
    return ";" if first_line.count(";") > first_line.count(",") else ","

def robust_read_csv(path: Path) -> pd.DataFrame:
    # try common encodings, guess separator from first line
    raw = path.read_bytes()
    for enc in ("utf-8-sig", "utf-8", "cp1252", "latin1"):
        try:
            txt = raw.decode(enc, errors="strict")
            sep = detect_sep(txt.splitlines()[0] if txt else ",")
            return pd.read_csv(io.StringIO(txt), sep=sep)
        except Exception:
            continue
    # last fallback
    txt = raw.decode("utf-8", errors="ignore")
    sep = detect_sep(txt.splitlines()[0] if txt else ",")
    return pd.read_csv(io.StringIO(txt), sep=sep)

def norm(s):
    # trim, remove non-breaking spaces, collapse internal spaces
    s = str(s).replace("\xa0", " ")
    s = re.sub(r"\s+", " ", s)
    return s.strip()
    
def normalize_text(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
    """Standardize smart quotes/whitespace in selected text cols."""
    repl = {
        "\u2018": "'", "\u2019": "'", "\u201C": '"', "\u201D": '"',
        "\u00A0": " ", "\u200B": ""
    }
    for c in cols:
        if c in df.columns:
            s = df[c].astype(str)
            for k,v in repl.items(): s = s.str.replace(k, v, regex=False)
            df[c] = s.str.strip()
    return df
    
# load survey answers
survey = robust_read_csv(SURVEY_FILE)
survey.columns = [norm(c) for c in survey.columns]

# add a simple participant id if missing
if "participant_id" not in survey.columns:
    survey.insert(0, "participant_id", [f"R{i+1:03d}" for i in range(len(survey))])

# drop trailing language question if present
if survey.columns[-1].lower().startswith("is your first language"):
    survey = survey.iloc[:, :-1]

# load blockmap
bm = robust_read_csv(BLOCKMAP_FILE)

# pick the question text column name
if "question" in bm.columns:
    bm = bm.rename(columns={"question": "question_text"})
elif "column" in bm.columns:
    bm = bm.rename(columns={"column": "question_text"})
else:
    raise ValueError("surveyblock_map must have 'question' or 'column' for the question text")

# normalize key text columns if they exist
for c in ["question_text", "construct", "Prompt_Id"]:
    if c in bm.columns:
        bm[c] = bm[c].map(norm)

# make sure flag columns exist and are numeric
for c in ["reverse", "is_filler", "is_attention"]:
    if c not in bm.columns:
        bm[c] = 0
    bm[c] = pd.to_numeric(bm[c], errors="coerce").fillna(0).astype(int)

# infer condition from Prompt_Id if blockmap has no condition column
if "condition" in bm.columns:
    bm["condition"] = bm["condition"].astype(str).str.upper().str.strip()
else:
    bm["condition"] = bm["Prompt_Id"].astype(str).str.extract(r"([AB])$", expand=False)

# reshape answers to long format
ans_long = survey.melt(id_vars=["participant_id"], var_name="question_text", value_name="response")
ans_long["question_text"] = ans_long["question_text"].map(norm)
ans_long["response"] = pd.to_numeric(ans_long["response"], errors="coerce")

# make a soft-normalized text key on both sides
def soft_norm(s):
    s = str(s).lower()
    s = re.sub(r"[^\w\s]", " ", s)     # remove punctuation
    s = re.sub(r"\s+", " ", s).strip() # collapse spaces
    return s

# blockmap normalize
bm["question_text"] = bm["question_text"].map(norm)
bm["q_norm"] = bm["question_text"].map(soft_norm)

# answers normalize
ans_long["q_norm"] = ans_long["question_text"].map(soft_norm)

# primary join by soft-normalized text
df = ans_long.merge(bm, on="q_norm", how="left", suffixes=("", "_bm"))

# if some rows still missing construct, try a light keyword rule using the question text
missing = df["construct"].isna()
if missing.any():
    qt = df.loc[missing, "question_text"].str.lower()

    # simple keyword hints
    is_csat = qt.str.contains("satisf")
    is_hl   = qt.str.contains("next step") | qt.str.contains("clear")
    is_cmp  = qt.str.contains("would follow") | qt.str.contains("would do")
    is_pi   = qt.str.contains("polite") | qt.str.contains("courteous") | qt.str.contains("tone") | qt.str.contains("please") | qt.str.contains("could")

    # fill construct from keywords when available
    df.loc[missing & is_csat, "construct"] = "CSAT"
    df.loc[missing & is_hl,   "construct"] = "HL"
    df.loc[missing & is_cmp,  "construct"] = "CMP"
    df.loc[missing & is_pi,   "construct"] = "PI"

    # also bring over Prompt_Id and condition by matching the A#/B# id
    df["col_id"] = df["question_text"].str.extract(r"\b([AB]\d{1,2})\b")
    bm2 = bm.copy()
    bm2["col_id"] = bm2["question_text"].str.extract(r"\b([AB]\d{1,2})\b")
    bm2 = bm2.dropna(subset=["col_id"])
    # merge only the id-based keys for rows still missing Prompt_Id/condition
    id_merge = df.loc[missing, ["q_norm","col_id"]].merge(
        bm2[["col_id","Prompt_Id","condition"]],
        on="col_id",
        how="left"
    )
    df.loc[missing, "Prompt_Id"] = df.loc[missing, "Prompt_Id"].where(df.loc[missing, "Prompt_Id"].notna(), id_merge["Prompt_Id"].values)
    if "condition" not in df.columns:
        df["condition"] = np.nan
    df.loc[missing, "condition"] = df.loc[missing, "condition"].where(df.loc[missing, "condition"].notna(), id_merge["condition"].values)

# standardize prompt id and condition
m = df["Prompt_Id"].astype(str).str.upper().str.extract(r"^(P\d{2,3})[_-]?([AB])?$")
df["Prompt_Id"] = m[0]
cond = df.get("condition").astype(str).str.upper().str.strip()
cond = np.where(cond.isin(["A","B"]), cond, m[1])
df["condition"] = cond

# keep only labeled rows now
df = df[df["construct"].notna()].copy()

# drop filler items
if "is_filler" in df.columns:
    df = df[df["is_filler"] != 1]

# drop participants who fail attention check
if (df["construct"].astype(str).str.upper() == "ATT_CHECK").any():
    att = (
        df[df["construct"].astype(str).str.upper() == "ATT_CHECK"]
        .groupby("participant_id")["response"]
        .apply(list).reset_index(name="att")
    )
    def passed(xs):
        xs = [v for v in xs if pd.notna(v)]
        return int(len(xs) > 0 and all(v == 5 for v in xs))
    att["pass"] = att["att"].apply(passed)
    df = df.merge(att[["participant_id", "pass"]], on="participant_id", how="left")
    df = df[df["pass"] == 1].drop(columns=["pass"])

# map construct names to canonical set
canon = {
    "PI": "PI", "POLITENESS": "PI",
    "CSAT": "CSAT", "SATISFACTION": "CSAT", "SAT": "CSAT",
    "HL": "HL", "HELPFULNESS": "HL", "CLARITY": "HL",
    "CMP": "CMP", "COMPLIANCE": "CMP"
}
df["construct"] = df["construct"].astype(str).str.upper().map(lambda x: canon.get(x, x))

# keep target constructs
targets = {"PI","CSAT","HL","CMP"}
df = df[df["construct"].isin(targets)].copy()
assert (df["construct"] == "CSAT").any(), "CSAT missing after merge/filter"

print("construct counts:", df["construct"].value_counts().to_dict())

# fallback merge by A/B item id for rows that did not match by text
# safer approach: build a unique mapping and left-join, then fill missing fields

# make column id in df for all rows
df["col_id"] = df["question_text"].str.extract(r"\b([AB]\d{1,2})\b")

# make a unique mapping table from blockmap
bm2 = bm.copy()
bm2["col_id"] = bm2["question_text"].str.extract(r"\b([AB]\d{1,2})\b")
bm2 = bm2[["col_id", "construct", "reverse", "Prompt_Id", "condition"]]
bm2 = bm2.dropna(subset=["col_id"]).drop_duplicates(subset=["col_id"], keep="first")

# join mapping onto df (suffix _byid)
df = df.merge(bm2, on="col_id", how="left", suffixes=("", "_byid"))

# fill only where original is missing
for c in ["construct", "reverse", "Prompt_Id", "condition"]:
    filler = f"{c}_byid"
    if filler in df.columns:
        df[c] = df[c].where(df[c].notna(), df[filler])

# cleanup helper columns
drop_cols = [c for c in ["col_id", "construct_byid", "reverse_byid", "Prompt_Id_byid", "condition_byid"] if c in df.columns]
df.drop(columns=drop_cols, inplace=True)

# canonical construct labels + keep targets
canon = {
    "PI":"PI","POLITENESS":"PI",
    "CSAT":"CSAT","SATISFACTION":"CSAT","SAT":"CSAT",
    "HL":"HL","HELPFULNESS":"HL","CLARITY":"HL",
    "CMP":"CMP","COMPLIANCE":"CMP"
}
targets = {"PI","CSAT","HL","CMP"}
df["construct"] = df["construct"].astype(str).str.upper().map(lambda x: canon.get(x, x))
df = df[df["construct"].isin(targets)].copy()
assert (df["construct"] == "CSAT").any(), "CSAT missing after merge/filter"

# reverse-score ONCE using the final 'reverse' flags
df["score"] = df["response"]
df.loc[df["reverse"] == 1, "score"] = 6 - df.loc[df["reverse"] == 1, "score"]

# drop filler
if "is_filler" in df.columns:
    df = df[df["is_filler"] != 1]

# attention-check (use raw responses; attention items must be 5)
if (df["construct"] == "ATT_CHECK").any():
    att = (df[df["construct"] == "ATT_CHECK"]
           .groupby("participant_id")["response"].apply(list).reset_index(name="att"))
    def passed(xs):
        xs = [v for v in xs if pd.notna(v)]
        return int(len(xs) > 0 and all(v == 5 for v in xs))
    att["pass"] = att["att"].apply(passed)
    df = df.merge(att[["participant_id","pass"]], on="participant_id", how="left")
    df = df[df["pass"] == 1].drop(columns=["pass"])

# standardize prompt id and condition
m = df["Prompt_Id"].astype(str).str.upper().str.extract(r"^(P\d{2,3})[_-]?([AB])?$")
df["Prompt_Id"] = m[0]
cond = df.get("condition").astype(str).str.upper().str.strip()
cond = np.where(cond.isin(["A", "B"]), cond, m[1])
df["condition"] = cond

# keep only labeled rows
df = df[df["construct"].notna()].copy()

# reverse score items marked as reverse (Likert 1..5 -> 6-x)
df.loc[df["reverse"] == 1, "response"] = 6 - df.loc[df["reverse"] == 1, "response"]

# drop filler items
df = df[df["is_filler"] != 1]

# drop participants who fail attention check (all attention items must be 5)
if (df["construct"].str.upper() == "ATT_CHECK").any():
    att = (
        df[df["construct"].str.upper() == "ATT_CHECK"]
        .groupby("participant_id")["response"]
        .apply(list)
        .reset_index(name="att")
    )
    def passed(xs):
        xs = [v for v in xs if pd.notna(v)]
        return int(len(xs) > 0 and all(v == 5 for v in xs))
    att["pass"] = att["att"].apply(passed)
    df = df.merge(att[["participant_id", "pass"]], on="participant_id", how="left")
    df = df[df["pass"] == 1].drop(columns=["pass"])

# keep target constructs
targets = {"PI", "CSAT", "HL", "CMP"}
df["construct"] = df["construct"].str.upper()
df = df[df["construct"].isin(targets)].copy()

# quick count by construct to confirm CSAT is present
print("construct counts:", df["construct"].value_counts().to_dict())

# aggregate to prompt x condition x construct
scored = (
    df.groupby(["Prompt_Id", "condition", "construct"])["response"]
      .mean()
      .reset_index()
)
wide = (
    scored.pivot(index=["Prompt_Id", "condition"], columns="construct", values="response")
          .reset_index()
)
wide.columns.name = None
for c in targets:
    if c in wide.columns:
        wide[c] = wide[c].round(3)

# try to merge task metadata if available
try:
    tasks = json.loads(Path(TASKS_FILE).read_text(encoding="utf-8"))
    tdf = pd.json_normalize(tasks)
    if "meta.prompt_id" not in tdf.columns and "meta.Prompt_Id" in tdf.columns:
        tdf = tdf.rename(columns={"meta.Prompt_Id": "meta.prompt_id",
                                  "meta.Reply_Id": "meta.reply_id"})
    tdf["Prompt_Id"] = tdf["meta.prompt_id"].astype(str).str.upper()
    tdf["condition"] = tdf["meta.condition"].astype(str).str.upper()
    tdf["reply_id"] = tdf["meta.reply_id"].astype(str)
    tdf["bot_mask"] = tdf.get("data.bot_mask", np.nan)
    tdf["reply_text"] = tdf.get("data.reply_text", np.nan)
    tdf = tdf[["Prompt_Id", "condition", "reply_id", "bot_mask", "reply_text"]].drop_duplicates()
    out = wide.merge(tdf, on=["Prompt_Id", "condition"], how="left")
except Exception as e:
    print("tasks_data.json merge skipped:", e)
    out = wide.copy()

# metadata loader: normalize Replies-masked.csv to expected columns
import pandas as pd

def load_metadata(path="Replies-masked.csv"):
    """
    Normalize reply metadata to:
      Prompt_Id (P##), condition ('A'/'B'), reply_id, bot_mask, reply_text.
    Robust to encoding/sep, header variants, and missing 'condition'.
    """
    df = robust_read_csv(Path(path))  # your existing robust reader

    # normalize header names
    def clean_name(s: str) -> str:
        s = str(s).strip().lower()
        s = re.sub(r"\s+", "_", s)
        s = s.replace("‐", "-")  # hyphen variants
        return s

    df.columns = [clean_name(c) for c in df.columns]

    # candidates by heuristic
    col_prompt = next((c for c in df.columns if c in {"prompt_id","prompt","promptid","p_id"} or "prompt" in c), None)
    col_cond   = next((c for c in df.columns if c in {"condition","cond","ab"} or "cond" in c), None)
    col_rid    = next((c for c in df.columns if c in {"reply_id","replyid","rid"} or ("reply" in c and "id" in c)), None)
    col_rtext  = next((c for c in df.columns if c in {"reply_text","reply","response_text"} or ("reply" in c and "text" in c)), None)
    col_bot    = next((c for c in df.columns if c in {"bot_mask","bot","botname"} or ("bot" in c and "mask" in c)), None)

    # rename to canonical
    ren = {}
    if col_prompt: ren[col_prompt] = "Prompt_Id"
    if col_cond:   ren[col_cond]   = "condition"
    if col_rid:    ren[col_rid]    = "reply_id"
    if col_rtext:  ren[col_rtext]  = "reply_text"
    if col_bot:    ren[col_bot]    = "bot_mask"
    if ren: df = df.rename(columns=ren)

    # type/casing cleanup
    for c in ["Prompt_Id","condition","reply_id"]:
        if c not in df: df[c] = pd.NA
        df[c] = df[c].astype(str).str.strip().str.upper()
    for c in ["bot_mask","reply_text"]:
        if c not in df: df[c] = pd.NA
        df[c] = df[c].astype(str)

    # infer condition if missing/empty: try Prompt_Id like "P03_A" or "P03A"
    if "condition" not in df or df["condition"].replace({"": pd.NA}).isna().all():
        pid = df.get("Prompt_Id", pd.Series([], dtype=str)).astype(str).str.upper()
        cond = pid.str.extract(r"(?:^|[_-])(A|B)(?:$|[_-])", expand=False)
        # fallback: last A/B in the string if pattern fail
        cond2 = pid.str.findall(r"[AB]").apply(lambda xs: xs[-1] if len(xs) else pd.NA)
        df["condition"] = cond.fillna(cond2)

    # as a second fallback, infer from any column that only contains A/B
    if df["condition"].isna().any():
        ab_cols = [c for c in df.columns if set(df[c].dropna().astype(str).str.upper().unique()) <= {"A","B"}]
        if ab_cols:
            df.loc[df["condition"].isna(), "condition"] = df.loc[df["condition"].isna(), ab_cols[0]].astype(str).str.upper()

    # final standardization
    df["Prompt_Id"] = df["Prompt_Id"].str.upper().str.extract(r"(P\d{2,3})", expand=False)
    df["condition"] = df["condition"].str.upper().where(df["condition"].isin(["A","B"]))

    # normalize smart quotes/nbsp
    df = normalize_text(df, ["Prompt_Id","condition","reply_text","bot_mask"])

    keep = ["Prompt_Id","condition","reply_id","bot_mask","reply_text"]
    return df[keep].drop_duplicates()
  
# --- metadata QA: require BOTH A and B per prompt, then check identical texts ---
meta = load_metadata("Replies-masked.csv")
meta = normalize_text(meta, ["Prompt_Id","condition","reply_text","bot_mask"])
meta["Prompt_Id"] = meta["Prompt_Id"].astype(str).str.upper().str.extract(r"(P\d{2,3})", expand=False)
meta["condition"] = meta["condition"].astype(str).str.upper().str.strip()
meta = meta[meta["condition"].isin(["A","B"])]

cond_sets = meta.groupby("Prompt_Id")["condition"].apply(lambda s: set(s.dropna()))
lacking = cond_sets[~cond_sets.apply(lambda s: {"A","B"}.issubset(s))].index.tolist()
assert not lacking, f"prompts without both A and B: {lacking}"

pairs = (meta.pivot_table(index="Prompt_Id", columns="condition",
                          values="reply_text", aggfunc="first")
           .reindex(columns=["A","B"])
           .dropna(subset=["A","B"]))
ident_prompts = pairs.index[pairs["A"] == pairs["B"]].tolist()
assert not ident_prompts, f"identical A/B replies for: {ident_prompts}"
print("metadata QA OK")

# merge metadata into scores at the end
out = wide.merge(meta, on=["Prompt_Id","condition"], how="left")

# save outputs
out.to_csv(OUT_WIDE, index=False, encoding="utf-8-sig")
df.to_csv(OUT_LONG, index=False, encoding="utf-8-sig")

with pd.ExcelWriter(OUT_XLSX) as xw:
    out.to_excel(xw, sheet_name="scores", index=False)
    df.head(5000).to_excel(xw, sheet_name="answers_long_preview", index=False)

print("saved:", OUT_WIDE, "columns:", list(out.columns))
print("saved:", OUT_LONG, "rows:", len(df))
print("construct counts:", df["construct"].astype(str).str.upper().value_counts().to_dict())
print("unique Prompt_Id:", df["Prompt_Id"].nunique(), "unique conditions:", df["condition"].nunique())


construct counts: {'PI': 7000, 'HL': 2100, 'CMP': 2100, 'CSAT': 2100}
construct counts: {'PI': 7000, 'HL': 2100, 'CMP': 2100, 'CSAT': 2100}
metadata QA OK
saved: processed_scores.csv columns: ['Prompt_Id', 'condition', 'CMP', 'CSAT', 'HL', 'PI', 'reply_id', 'bot_mask', 'reply_text']
saved: answers_long.csv rows: 13300
construct counts: {'PI': 7000, 'HL': 2100, 'CMP': 2100, 'CSAT': 2100}
unique Prompt_Id: 5 unique conditions: 2


In [20]:
# A vs B (independent) analysis from answers_long.csv
# Saves per-prompt Welch tests and an across-prompt summary with 3dp rounding.

import pandas as pd
import numpy as np
from scipy import stats
from pathlib import Path

IN_LONG = Path("answers_long.csv")
OUT_PER_PROMPT = Path("ab_welch_per_prompt.csv")
OUT_SUMMARY = Path("ab_summary_overall.csv")

# load and filter
df = pd.read_csv(IN_LONG, encoding="utf-8-sig")
metrics = ["PI", "CSAT", "HL", "CMP"]
df = df[df["construct"].isin(metrics)].copy()
df["condition"] = df["condition"].astype(str).str.upper().str.strip()
df = df[df["condition"].isin(["A", "B"])].copy()
df["response"] = pd.to_numeric(df["response"], errors="coerce")

# collapse to participant means per Prompt × Condition × Metric
dfp = (df.groupby(["participant_id", "Prompt_Id", "condition", "construct"])["response"]
         .mean()
         .reset_index()
         .rename(columns={"construct": "metric"}))

def welch_test(a, b):
    """
    Welch's t on independent samples a vs b.
    Returns stats with diff = mean(b) - mean(a), and t aligned to that diff.
    """
    a = pd.Series(a, dtype="float64").dropna()
    b = pd.Series(b, dtype="float64").dropna()
    n1, n2 = len(a), len(b)
    if n1 < 2 or n2 < 2:
        return dict(nA=n1, nB=n2, meanA=a.mean(), meanB=b.mean(), diff=b.mean() - a.mean(),
                    t=np.nan, df=np.nan, p=np.nan, d=np.nan, g=np.nan,
                    ci_low=np.nan, ci_high=np.nan)

    # order matters: ttest_ind(b, a) so t matches diff = B - A
    t, p = stats.ttest_ind(b, a, equal_var=False, nan_policy="omit")

    s1, s2 = a.var(ddof=1), b.var(ddof=1)
    se = np.sqrt(s1/n1 + s2/n2)

    df_w = (s1/n1 + s2/n2) ** 2 / ((s1**2) / (n1**2 * (n1 - 1)) + (s2**2) / (n2**2 * (n2 - 1)))
    tcrit = stats.t.ppf(0.975, df=df_w) if np.isfinite(df_w) else np.nan

    sp = np.sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))
    d = (b.mean() - a.mean()) / sp if np.isfinite(sp) and sp > 0 else np.nan
    J = 1 - 3 / (4 * (n1 + n2) - 9) if (n1 + n2) > 3 else np.nan
    g = d * J if np.isfinite(d) and np.isfinite(J) else np.nan

    diff = b.mean() - a.mean()
    ci_low = diff - tcrit * se if np.isfinite(tcrit) else np.nan
    ci_high = diff + tcrit * se if np.isfinite(tcrit) else np.nan

    return dict(nA=n1, nB=n2, meanA=a.mean(), meanB=b.mean(), diff=diff,
                t=t, df=df_w, p=p, d=d, g=g, ci_low=ci_low, ci_high=ci_high)

# per-prompt Welch tests on participant means
rows = []
for m in metrics:
    sub = dfp[dfp["metric"] == m]
    for pid, grp in sub.groupby("Prompt_Id"):
        a = grp.loc[grp["condition"] == "A", "response"]
        b = grp.loc[grp["condition"] == "B", "response"]
        res = welch_test(a, b)
        res.update(metric=m, Prompt_Id=pid)
        rows.append(res)

per_prompt = pd.DataFrame(rows)
per_prompt = per_prompt[["metric","Prompt_Id","nA","nB","meanA","meanB","diff","t","df","p","d","g","ci_low","ci_high"]]
per_prompt = per_prompt.sort_values(["metric","Prompt_Id"]).reset_index(drop=True)

# round for output
pp_cols = ["meanA","meanB","diff","t","df","p","d","g","ci_low","ci_high"]
per_prompt[pp_cols] = per_prompt[pp_cols].round(3)
per_prompt.to_csv(OUT_PER_PROMPT, index=False, encoding="utf-8-sig")

# across-prompt summary: 1-sample t on per-prompt diffs
sum_rows = []
for m in metrics:
    dvals = per_prompt.loc[per_prompt["metric"] == m, "diff"].astype(float).dropna()
    if dvals.empty:
        sum_rows.append(dict(metric=m, k_prompts=0, mean_diff=np.nan, t=np.nan, p=np.nan,
                             ci_low=np.nan, ci_high=np.nan, d_z=np.nan))
        continue
    t1, p1 = stats.ttest_1samp(dvals, 0.0, nan_policy="omit")
    mean_diff = dvals.mean()
    sd = dvals.std(ddof=1) if len(dvals) > 1 else np.nan
    se = sd/np.sqrt(len(dvals)) if np.isfinite(sd) and sd > 0 else np.nan
    tcrit = stats.t.ppf(0.975, df=len(dvals)-1) if len(dvals) > 1 else np.nan
    ci_low = mean_diff - tcrit*se if np.isfinite(tcrit) else np.nan
    ci_high = mean_diff + tcrit*se if np.isfinite(tcrit) else np.nan
    dz = mean_diff / sd if np.isfinite(sd) and sd > 0 else np.nan
    sum_rows.append(dict(metric=m, k_prompts=len(dvals), mean_diff=round(mean_diff,3),
                         t=t1, p=p1, ci_low=ci_low, ci_high=ci_high, d_z=dz))

summary = pd.DataFrame(sum_rows)
summary[["t","p","ci_low","ci_high","d_z"]] = summary[["t","p","ci_low","ci_high","d_z"]].round(3)
summary.to_csv(OUT_SUMMARY, index=False, encoding="utf-8-sig")

print("saved", OUT_PER_PROMPT, "and", OUT_SUMMARY)
print(summary)


saved ab_welch_per_prompt.csv and ab_summary_overall.csv
  metric  k_prompts  mean_diff      t      p  ci_low  ci_high    d_z
0     PI          5     -0.192 -3.728  0.020  -0.335   -0.049 -1.667
1   CSAT          3     -0.092 -0.907  0.460  -0.527    0.343 -0.523
2     HL          3      0.095  0.483  0.677  -0.751    0.941  0.279
3    CMP          3     -0.411 -1.141  0.372  -1.962    1.140 -0.659
